In [1]:
import sys
sys.path.append('..')
from NEMtropy import *
# import bicm
# from bicm import *
import numpy as np

This test uses a plant-pollinator network from Petanidou, T. (1991). Pollination ecology in a phryganic ecosystem.
This script is run on an ordinary laptop.

In [2]:
biad_mat_names = np.loadtxt('test_dataset_bicm.csv', delimiter=',', dtype=str)
plants = biad_mat_names[1:,0]
pollinators = biad_mat_names[0, 1:]
biad_mat = biad_mat_names[1:, 1:].astype(np.ubyte)
plants_dict = dict(enumerate(np.unique(plants)))
plants_inv_dict = {v:k for k,v in plants_dict.items()}
pollinators_dict = dict(enumerate(np.unique(pollinators)))
pollinators_inv_dict = {v:k for k,v in pollinators_dict.items()}

In [3]:
from scipy.sparse import *

In [4]:
sp_mat = csr_matrix(biad_mat)

In [5]:
bg = BipartiteGraph(sp_mat)

In [6]:
bg.biadjacency

<131x666 sparse matrix of type '<class 'numpy.uint8'>'
	with 2933 stored elements in Compressed Sparse Row format>

In [7]:
red_mat = bg.initialize_avg_mat()

In [8]:
bg.solve_tool()

/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm-3.1-py3.10.egg/bicm/models_functions.py:326: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm-3.1-py3.10.egg/bicm/models_functions.py:327: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]


max rows error = 7.144598246355827e-10
max columns error = 2.1845778519491432e-09
total error = 4.569068901005835e-09
Solver converged.


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


You can manipulate a data structure into another with the functions of the package. It supports biadjacency matrices (numpy arrays or scipy sparse matrices), edgelists, adjacency lists (dictionaries) or just the degree sequences.

Let's create an edgelist with names.

In [9]:
# These data type converters return additional objects, keeping track of the node labeling and the node degrees.
# The desired data structure is always at index 0. Check the documentation for more info.
edgelist = edgelist_from_biadjacency(biad_mat)[0]
edgelist_names = [(plants_dict[edge[0]], pollinators_dict[edge[1]]) for edge in edgelist]
print(edgelist_names[:5])

[('"Acanthus spinosus"', '"Acmaeodera bipunctata "'), ('"Acanthus spinosus"', '"Adia cinerella "'), ('"Acanthus spinosus"', '"Alophora pusilla"'), ('"Acanthus spinosus"', '"Amasis lateralis"'), ('"Acanthus spinosus"', '"Amasis similis"')]


Now we can use the edgelist to create the bipartite graph. In two ways:

In [10]:
myGraph = BipartiteGraph(edgelist=edgelist_names)

In [11]:
myGraph = BipartiteGraph()
myGraph.set_edgelist(edgelist_names)

And now let's simply compute the bicm! This should run instantly. The solver checks that the solution is correct automatically.

In [12]:
myGraph.solve_tool()
dict_x, dict_y = myGraph.get_bicm_fitnesses()
print('Yielded data type is:', type(dict_x))

max rows error = 7.144598246355827e-10
max columns error = 2.1845778519491432e-09
total error = 4.569068901005835e-09
Solver converged.
Yielded data type is: <class 'dict'>


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


The computed fitnesses are given as dictionaries, keeping track of the name of the nodes. If you are sure about the order of the fitnesses, you can get separately the fitness vectors and the dictionaries that keep track of the order of the nodes:

In [13]:
x = myGraph.x
y = myGraph.y
rows_dict = myGraph.rows_dict
cols_dict = myGraph.cols_dict

Alternatively, you can have the bicm matrix of probabilities, whose entries i, j are the bicm probabilities of having a link between nodes i and j (who will correspond to rows_dict[i] and cols_dict[j]), or compute it on your own

In [14]:
avg_mat = myGraph.get_bicm_matrix()
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


In [15]:
avg_mat = bicm_from_fitnesses(x, y)
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


And, if you like, you can check the solution by yourself:

In [16]:
# The data type conversions are generally consistent with the order, so this works
myGraph.check_sol(biad_mat, avg_mat)

max rows error = 7.144596025909777e-10
max columns error = 2.1846062736585736e-09
total error = 9.57038759352713e-09


You can then sample from the BiCM ensemble in two ways, via matrix sampling or edgelist sampling.

In [17]:
# Here we generate a sampling of 1000 networks and calculate the distribution of the number of links.
tot_links = []
for sample_i in range(1000):
    sampled_network = sample_bicm(avg_mat)
    tot_links.append(np.sum(sampled_network))

In [18]:
sampled_edgelist = sample_bicm_edgelist(x, y)

Now let's compute the projection: it can take some seconds.

In [19]:
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 566.11it/s]

{'"Astragalus monspessulanus"': ['"Biscutella didyma"'], '"Asphodelus aestivus"': ['"Astragalus monspessulanus"', '"Biscutella didyma"'], '"Heliotropium dolosum"': ['"Hypochoeris achyrophorus"'], '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'], '"Echinops microcephalus"': ['"Fumana thymifolia"'], '"Carduus pycnocephalus"': ['"Hirschfeldia incana"'], '"Fritillaria graeca"': ['"Heliotropium dolosum"', '"Hypochoeris achyrophorus"'], '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'], '"Fumana arabica"': ['"Papaver rhoeas"'], '"Plantago lagopus"': ['"Prasium majus"']}


If you don't want to get the progress_bar you can set progress_bar=False. If you want to re-compute the projection with different settings (here a lower validation threshold), use compute_projection()

In [20]:
myGraph.compute_projection(rows=True, alpha=0.01, progress_bar=False)

In [21]:
# You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"
print('You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"')

rows_projection = myGraph.get_rows_projection(fmt='edgelist')
print(rows_projection)

You can ask for an edgelist instead of an adjacency list by setting fmt="edgelist"
[['"Astragalus monspessulanus"' '"Biscutella didyma"']
 ['"Asphodelus aestivus"' '"Astragalus monspessulanus"']
 ['"Asphodelus aestivus"' '"Biscutella didyma"']
 ['"Centaurea raphanina"' '"Echinops microcephalus"']
 ['"Centaurea raphanina"' '"Fumana thymifolia"']
 ['"Echinops microcephalus"' '"Fumana thymifolia"']
 ['"Phlomis fructicosa"' '"Plantago lagopus"']
 ['"Phlomis fructicosa"' '"Prasium majus"']
 ['"Fumana arabica"' '"Papaver rhoeas"']
 ['"Plantago lagopus"' '"Prasium majus"']]


These projection only contain links between nodes that behave similarly with respect to the expected behavior calculated from the BiCM. They could also be empty:

In [22]:
cols_projection = myGraph.get_cols_projection()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666/666 [00:00<00:00, 792.70it/s]


No V-motifs will be validated. Try increasing alpha


In [23]:
# Increasing alpha too much will make the projection too dense. 
# It is not suggested: if it is empty with a high alpha, try different projections.
# Check the documentation and the papers for more info.
myGraph.compute_projection(rows=False, alpha=0.2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 666/666 [00:01<00:00, 454.79it/s]


No V-motifs will be validated. Try increasing alpha


Different projection methods are implemented. 
They differ in the way of approximating the poisson-binomial distribution of the similarities between nodes.
By default, the poisson approximation is used, since the poisson binomial is very expensive to compute. Check the docs for more!
Here, an example of the poibin implementation. This can take even some minutes, and it's faster when using a biadjacency matrix instead of other data types. Note that the poisson approximation is fairly good.

In [24]:
myGraph.compute_projection(rows=True, method='poibin')
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

"method" is deprecated, use approx_method instead
Calculating p-values...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1225/1225 [00:15<00:00, 79.07it/s]


{'"Astragalus monspessulanus"': ['"Biscutella didyma"'], '"Asphodelus aestivus"': ['"Astragalus monspessulanus"', '"Biscutella didyma"'], '"Heliotropium dolosum"': ['"Hypochoeris achyrophorus"'], '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'], '"Echinops microcephalus"': ['"Fumana thymifolia"'], '"Carduus pycnocephalus"': ['"Hirschfeldia incana"'], '"Fritillaria graeca"': ['"Heliotropium dolosum"', '"Hypochoeris achyrophorus"'], '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'], '"Ophrys tenthredinifera"': ['"Papaver rhoeas"'], '"Fumana arabica"': ['"Papaver rhoeas"'], '"Plantago lagopus"': ['"Prasium majus"'], '"Euphorbia acanthothamnos"': ['"Geranium rotundifolium"']}


Now, let's have fun with other data structures and solver methods.

In [25]:
adj_list_names = adjacency_list_from_edgelist_bipartite(edgelist_names, convert_type=False)[0]

In [26]:
myGraph2 = BipartiteGraph(adjacency_list=adj_list_names)

Asking for the projection immediately will do everything automatically.

In [27]:
myGraph2.get_rows_projection()

First I have to compute the BiCM. Computing...
max rows error = 7.144598246355827e-10
max columns error = 2.1845778519491432e-09
total error = 4.569068901005835e-09
Solver converged.


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 427.94it/s]


{'"Asphodelus aestivus"': ['"Astragalus monspessulanus"',
  '"Biscutella didyma"'],
 '"Astragalus monspessulanus"': ['"Biscutella didyma"'],
 '"Carduus pycnocephalus"': ['"Hirschfeldia incana"'],
 '"Centaurea raphanina"': ['"Echinops microcephalus"', '"Fumana thymifolia"'],
 '"Echinops microcephalus"': ['"Fumana thymifolia"'],
 '"Fritillaria graeca"': ['"Heliotropium dolosum"',
  '"Hypochoeris achyrophorus"'],
 '"Fumana arabica"': ['"Papaver rhoeas"'],
 '"Heliotropium dolosum"': ['"Hypochoeris achyrophorus"'],
 '"Phlomis fructicosa"': ['"Plantago lagopus"', '"Prasium majus"'],
 '"Plantago lagopus"': ['"Prasium majus"']}

In [28]:
rows_deg = biad_mat.sum(1)
cols_deg = biad_mat.sum(0)

In [29]:
myGraph3 = BipartiteGraph(degree_sequences=(rows_deg, cols_deg))

Default (recommended) method is 'newton', three more methods are implemented

In [30]:
myGraph3.solve_tool(method='fixed-point', tol=1e-5, exp=True)

max rows error = 7.971934223860444e-09
max columns error = 2.449649088021033e-07
total error = 3.4629645728756486e-07
Solver converged.


In [31]:
myGraph3.solve_tool(method='quasinewton', tol=1e-12, exp=True)

/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm-3.1-py3.10.egg/bicm/models_functions.py:404: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  step_fun = args[0]
/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/bicm-3.1-py3.10.egg/bicm/models_functions.py:405: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  arg_step_fun = args[1]


max rows error = 5.133965146342234e-07
max columns error = 4.904201205135905e-07
total error = 4.5348409931644795e-06
Solver converged.


/home/matteo/miniconda3/envs/bicm/lib/python3.10/site-packages/numba/core/utils.py:612: NumbaExperimentalFeatureWarning: First-class function type feature is experimental
  warnings.warn("First-class function type feature is experimental",


This is the only method that works with a scipy solver, not recommended.

In [32]:
myGraph3.solve_tool(method='root')

max rows error = 2.1316282072803006e-14
max columns error = 7.105427357601002e-15
total error = 2.0605739337042905e-13
Solver converged.


If you want to retrieve the p-values of the projection on one layer, just call the pvals on the object after the computation:

In [33]:
# Use the rows arg to project on the rows or columns
myGraph2.compute_projection()
pval_adj_list = myGraph2.rows_pvals

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 131/131 [00:00<00:00, 401.54it/s]


Keep in mind that the p-values are an adjacency list as well. You can now obtain the full matrix:

In [34]:
myGraph.get_projected_pvals_mat()

Please specify the layer with layer='rows' or layer='columns'.


In [35]:
myGraph.get_projected_pvals_mat(layer='columns')

array([[1.        , 0.47139158, 0.33940164, ..., 0.59288338, 0.59288338,
        1.        ],
       [0.47139158, 1.        , 0.03280358, ..., 1.        , 1.        ,
        0.36232276],
       [0.33940164, 0.03280358, 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [0.59288338, 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.59288338, 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 0.36232276, 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [36]:
myGraph.get_projected_pvals_mat(layer='rows')

array([[1.        , 0.99905974, 1.        , ..., 0.66448969, 0.42182595,
        0.42182595],
       [0.99905974, 1.        , 0.0942629 , ..., 0.56831711, 1.        ,
        0.3471088 ],
       [1.        , 0.0942629 , 1.        , ..., 0.55375732, 1.        ,
        0.33657006],
       ...,
       [0.66448969, 0.56831711, 0.55375732, ..., 1.        , 1.        ,
        0.07294914],
       [0.42182595, 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.42182595, 0.3471088 , 0.33657006, ..., 0.07294914, 1.        ,
        1.        ]])